# Анализ базы данных крупного сервиса для чтения книг по подписке

**Цель исследования:**
    
Проанализировать базу данных крупного сервиса для чтения книг по подписке и сформулировать предложение, которое поможет улучшить новый продукт.   

**Задачи исследования:**

1) Посчитайть, сколько книг вышло после 1 января 2000 года;

2) Для каждой книги посчитать количество обзоров и среднюю оценку;

3) Определить издательство, которое выпустило наибольшее число книг толще
   50 страниц — исключить из анализа брошюры;
   
4) Определить автора с самой высокой средней оценкой книг — учитывать
   только книги с 50 и более оценками;
   
5) Посчитать среднее количество обзоров от пользователей, которые поставили
   больше 48 оценок.

## Получение доступа к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Изучение общей информации по таблицам базы данных

In [2]:
def table_df(table):
    # выведем на экран первые пять строк таблицы 
    query = '''SELECT *
    FROM {} 
    LIMIT 5;
    '''.format(table)
    con=engine.connect()
    df = pd.io.sql.read_sql(sql=text(query), con = con)
    display(df)

In [3]:
for t in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print('Первые 5 строк таблицы:', t)
    table_df(t)

Первые 5 строк таблицы: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Первые 5 строк таблицы: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Первые 5 строк таблицы: publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Первые 5 строк таблицы: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Первые 5 строк таблицы: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [4]:
def table_info(table):
    # выведем на экран количество строк в таблице
    query = '''SELECT COUNT(*)
    FROM {};
    '''.format(table)
    con=engine.connect()
    df = pd.io.sql.read_sql(sql=text(query), con = con)
    display(df)

In [5]:
for t in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print('Количество строк в таблице:', t)
    table_info(t)

Количество строк в таблице: books


,count
0,1000


Количество строк в таблице: authors


,count
0,636


Количество строк в таблице: publishers


,count
0,340


Количество строк в таблице: ratings


,count
0,6456


Количество строк в таблице: reviews


,count
0,2793


 **Выводы по п.2:**

 В таблице books - 1000 строк и 6 столбцов, в которых содержится информация об идентификаторе книги, идентификаторе автора, название книги, количество страниц книги, дата публикации книги и идентификатор издателя.
 
 В таблице authors - 636 строк и 2 столбца, в которых содержится информация об идентификаторе автора и имя автора.
 
 В таблице publishers - 340 строк и 2 столбца, в которых содержится информация об идентификаторе издательства и название издательства.
 
 В таблице ratings - 6456 строк и 4 столбца, в которых содержится информация об идентификаторе оценки, идентификаторе книги, имя пользователя, оставившего оценку, оценка книги.
 
 В таблице reviews - 2793 строки  4 столбца, в которых содержится информация об идентификаторе обзора, идентификаторе книги, имя пользователя, написавшего обзор, текст обзора.

## Расчет количества книг, которые вышли после 1 января 2000 года

In [6]:
#посчитаем количество книг, которые вышли после 1 января 2000 года
query = '''SELECT COUNT(book_id) AS count_books
           FROM books
           WHERE publication_date >= '2000-01-01'
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count_books
0,821


**Выводы по п.3:**
    
821 книга вышла после 1 января 2000 года. Дату 1 января 2000 года включили, так как нелогично выкидывать из расчета за 2000 год и далее один день. 

## Расчет количества обзоров и средней оценки для каждой книги

In [7]:
#посчитаем общее количество обзоров в базе данных
query = '''SELECT COUNT(review_id)
           FROM reviews           
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


Общее количество обзоров в базе данных - 2793.

In [8]:
#посчитаем количества обзоров и средней оценки для каждой книги
query = '''SELECT b.book_id,
           b.title,
           COUNT(DISTINCT r.review_id) AS count_review,
           ROUND(AVG(rat.rating)) AS avg_rating
           FROM reviews AS r
           RIGHT JOIN books AS b ON b.book_id=r.book_id      
           LEFT JOIN ratings AS rat ON rat.book_id=b.book_id
           GROUP BY  b.book_id
           ORDER BY count_review DESC
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,4.0
1,963,Water for Elephants,6,4.0
2,734,The Glass Castle,6,4.0
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.0
4,695,The Curious Incident of the Dog in the Night-Time,6,4.0
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,4.0
996,808,The Natural Way to Draw,0,3.0
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.0
998,221,Essential Tales and Poems,0,4.0


**Выводы по п.4:**

Книги, которые имеют самое большое количество обзоров - "Twilight", "Water for Elephants", "The Glass Castle", "Harry Potter and the Prisoner of Azkaban", "The Curious Incident of the Dog in the Night-Time	". Каждая из них получила среднюю оценку пользователей: 4.

## Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
#определим издательство, которое выпустило наибольшее число книг толще 50 страниц
query = '''SELECT p.publisher,
           COUNT(b.book_id) AS count_books
           FROM publishers AS p
           JOIN books AS b ON b.publisher_id =p.publisher_id 
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY count_books DESC
           LIMIT 1;
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_books
0,Penguin Books,42


**Выводы по п.5:**
    
Издательство, которое выпустило наибольшее число книг больше 50 страниц - Penguin Books. Количество таких выпущенных книг у издательства - 42 шт. 

## Определение автора с самой высокой средней оценкой книг среди книг с 50 и более оценками

In [10]:
#определим автора с самой высокой средней оценкой книг среди книг с 50 и более оценками
query = '''SELECT a.author,
           ROUND(AVG(r.rating), 2) AS avg_rating
           FROM authors AS a
           JOIN books AS b ON b.author_id =a.author_id 
           JOIN ratings AS r ON b.book_id  =r.book_id 
           WHERE b.book_id IN (SELECT b.book_id
                               FROM books AS b
                               JOIN ratings AS r ON b.book_id  =r.book_id 
                               GROUP BY b.book_id
                               HAVING COUNT(r.rating) > 50)
           GROUP BY a.author
           ORDER BY avg_rating DESC
           LIMIT 1
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг среди книг с 50 и более оценками - J.K. Rowling/Mary GrandPré. Средний рейтинг его книг - 4.29.

## Среднее количество обзоров от пользователей, которые поставили более 48 оценок

In [11]:
#определим среднее количество обзоров от пользователей, которые поставили более 48 оценок
query = '''SELECT AVG(r1.count_review) AS avg_count_review
           FROM (SELECT r.username,
                 COUNT(r.review_id ) AS count_review
                 FROM reviews AS r
                 WHERE r.username IN (SELECT rat.username
                                    FROM ratings AS rat
                                    GROUP BY rat.username
                                    HAVING COUNT(rat.rating) > 48) 
                 GROUP BY r.username) AS r1                   
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_count_review
0,24.0


**Выводы по п.7:**

Среднее количество обзоров от пользователей, которые поставили более 48 оценок, - 24.0.

## Общий вывод по проекту

В работе над заданиями проекта использовалась база данных сервиса книг по подписке, в которой 5 таблиц.

В **таблице book** - 1000 строк и 6 столбцов, в которых содержится информация об идентификаторе книги, идентификаторе автора, название книги, количество страниц книги, дата публикации книги и идентификатор издателя.
 
 В **таблице authors** - 636 строк и 2 столбца, в которых содержится информация об идентификаторе автора и имя автора.
 
 В **таблице publishers** - 340 строк и 2 столбца, в которых содержится информация об идентификаторе издательства и название издательства.
 
 В **таблице ratings** - 6456 строк и 4 столбца, в которых содержится информация об идентификаторе оценки, идентификаторе книги, имя пользователя, оставившего оценку, оценка книги.
 
 В **таблице reviews** - 2793 строки  4 столбца, в которых содержится информация об идентификаторе обзора, идентификаторе книги, имя пользователя, написавшего обзор, текст обзора.
 
На основании выполненных исследований получили следующие выводы по базе данных:
 
 - 821 книга вышла после 1 января 2000 года. 

 - Книги, которые имеют самое большое количество обзоров - "Twilight", "Water for Elephants", "The Glass Castle", "Harry Potter and the Prisoner of Azkaban", "The Curious Incident of the Dog in the Night-Time	". Каждая из них получила среднюю оценку пользователей: 4.

 - Издательство, которое выпустило наибольшее число книг больше 50 страниц - Penguin Books. Количество таких выпущенных книг у издательства - 42 шт. 

- Автор с самой высокой средней оценкой книг среди книг с 50 и более оценками - J.K. Rowling/Mary GrandPré. Средний рейтинг его книг - 4.29.

- Среднее количество обзоров от пользователей, которые поставили более 48 оценок, - 24.0.